# Strategy Notebook Template

Use this notebook to document, visualise, and backtest a swing strategy (1D / intraday).


## Sections
- Hypothesis & market regime
- Indicators & parameters
- Entry / exit logic
- Position sizing (default 2% risk)
- Visual checks (price + signals)
- Backtest metrics
- Interactive review (entries/exits + equity)
- Robustness checklist


## 1. Hypothesis
Describe the inefficiency the strategy targets (pattern, regime, instruments).


## 2. Regime Filters
List the filters you will use (trend, volatility, market breadth, etc.).


In [1]:
from pathlib import Path
import sys

def find_project_root(start=None):
    start = Path.cwd() if start is None else Path(start)
    for candidate in [start] + list(start.parents):
        if (candidate / 'utils').exists() and (candidate / 'backtesting').exists():
            return candidate
    return start

PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

print('Project root:', PROJECT_ROOT)


Project root: c:\Users\saill\Desktop\t_project


In [2]:
# Notebook configuration (edit per strategy)
TICKER = 'SPY'
START_DATE = '2015-01-01'
END_DATE = '2025-01-01'

TIMEFRAME_DAILY = '1d'
TIMEFRAME_INTRADAY = '1h'
RESAMPLE_TO = '4H'

RISK_PCT = 0.02
DEFAULT_STOP_DISTANCE = 0.03


In [3]:
import pandas as pd

from utils.data_manager import DataManager
from utils.data_processor import resample_data

dm = DataManager()
df_daily = dm.get_data(TICKER, START_DATE, END_DATE, interval=TIMEFRAME_DAILY)
df_intraday = dm.get_data(TICKER, START_DATE, END_DATE, interval=TIMEFRAME_INTRADAY)

if df_intraday is not None and not df_intraday.empty:
    df_4h = resample_data(df_intraday, rule=RESAMPLE_TO)
else:
    df_4h = pd.DataFrame()
    print('Intraday data unavailable; continuing without 4H resample.')

print(f'Daily bars: {len(df_daily)}')
if not df_4h.empty:
    print(f'4H bars: {len(df_4h)}')


2025-11-03 01:49:55 - utils.data_manager - INFO - DataManager initialisé. Cache: C:\Users\saill\Desktop\t_project\data\cache. Timezone: Europe/Paris
2025-11-03 01:49:55 - utils.data_manager - INFO - Cache insuffisant pour SPY (demandé: 2015-01-01 à 2025-01-01). Re-téléchargement.
2025-11-03 01:49:55 - utils.data_manager - INFO - Téléchargement pour SPY (plage par défaut : 2015-01-01 à 2025-10-31)...
2025-11-03 01:49:55 - utils.data_manager - INFO - Téléchargement de SPY (2015-01-01 à 2025-10-31, 1d)...
2025-11-03 01:49:56 - utils.data_manager - INFO - Données téléchargées avec succès pour SPY (2724 lignes).
2025-11-03 01:49:56 - utils.data_manager - INFO - [OK] Données prêtes pour SPY (2516 lignes de 2015-01-01 à 2025-01-01).
2025-11-03 01:49:56 - utils.data_manager - INFO - Téléchargement pour SPY (plage par défaut : 2015-01-01 à 2025-10-31)...
2025-11-03 01:49:56 - utils.data_manager - INFO - Téléchargement de SPY (2015-01-01 à 2025-10-31, 1h)...
2025-11-03 01:49:56 - utils.data_mana

Intraday data unavailable; continuing without 4H resample.
Daily bars: 2516


In [4]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_plot = df_daily.copy()
try:
    import pandas_ta as ta
    df_plot['EMA_FAST'] = ta.ema(df_plot['close'], length=20)
    df_plot['EMA_SLOW'] = ta.ema(df_plot['close'], length=50)
    df_plot['RSI14'] = ta.rsi(df_plot['close'], length=14)
except Exception:
    df_plot['EMA_FAST'] = df_plot['close'].rolling(20).mean()
    df_plot['EMA_SLOW'] = df_plot['close'].rolling(50).mean()
    df_plot['RSI14'] = df_plot['close'].rolling(14).apply(lambda _ : 50.0)

if getattr(df_plot.index, 'tz', None) is not None:
    xs = df_plot.index.tz_convert('UTC').tz_localize(None)
else:
    xs = df_plot.index

fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])
fig.add_trace(go.Candlestick(x=xs, open=df_plot['open'], high=df_plot['high'], low=df_plot['low'], close=df_plot['close'], name='OHLC'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['EMA_FAST'], mode='lines', name='EMA_FAST'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['EMA_SLOW'], mode='lines', name='EMA_SLOW'), row=1, col=1)
fig.add_trace(go.Scatter(x=xs, y=df_plot['RSI14'], mode='lines', name='RSI14', line=dict(color='purple')), row=2, col=1)
fig.update_yaxes(title_text='Price', row=1, col=1)
fig.update_yaxes(title_text='RSI', row=2, col=1, range=[0, 100])
fig.update_layout(title=f'{TICKER} quick visual check', xaxis_rangeslider_visible=False, template='plotly_white')
fig.show()


## 3. Entry / Exit Logic
Document the exact boolean rules for entries, exits, and invalidation.


In [5]:
import importlib
import backtrader as bt

from backtesting.engine import BacktestEngine
from risk_management.position_sizing import FixedFractionalSizer
from utils.config_loader import get_settings

STRATEGY_PATH = 'strategies.implementations.simple_ma_managed_strategy.SimpleMaManagedStrategy'  # update
STRATEGY_PARAMS = {}  # override defaults here

module_name, class_name = STRATEGY_PATH.rsplit('.', 1)
StrategyClass = getattr(importlib.import_module(module_name), class_name)

engine = BacktestEngine()
engine.cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='timereturn')
engine.add_sizer(FixedFractionalSizer, risk_pct=RISK_PCT, stop_distance=DEFAULT_STOP_DISTANCE)
engine.add_data(df_daily, name='data_1d')
if not df_4h.empty:
    engine.add_data(df_4h, name='data_4h')

engine.add_strategy(StrategyClass, **STRATEGY_PARAMS)
results = engine.run()
strategy_instance = results[0]


2025-11-03 01:50:07 - backtesting.engine - INFO - Initialisation du BacktestEngine...
2025-11-03 01:50:07 - backtesting.engine - INFO - Capital initial du broker fixé à : 10,000.00
2025-11-03 01:50:07 - backtesting.engine - INFO - Commission (pourcentage) fixée à : 0.1000%
2025-11-03 01:50:07 - backtesting.engine - INFO - Slippage (pourcentage) fixé à : 0.0500%
2025-11-03 01:50:07 - backtesting.engine - INFO - Position Sizer 'FixedFractionalSizer' ajouté. Paramètres: (risk_pct=0.02, stop_distance=0.03)
2025-11-03 01:50:07 - backtesting.engine - INFO - Flux de données 'data_1d' ajouté. Période: 2015-01-02 à 2024-12-31.
2025-11-03 01:50:07 - backtesting.engine - INFO - Stratégie 'SimpleMaManagedStrategy' ajoutée. Paramètres: ()
2025-11-03 01:50:07 - backtesting.engine - INFO - --- DÉMARRAGE DU BACKTEST ---
2025-11-03 01:50:10 - strategies.implementations.simple_ma_managed_strategy - INFO - SimpleMaManagedStrategy initialisée - Fast MA: 10, Slow MA: 30
2025-11-03 01:50:10 - strategies.imp

In [6]:
initial_capital = get_settings().get('backtest', {}).get('initial_capital', 10000.0)
try:
    from scripts.run_backtest import print_results
    print_results(results, initial_capital, df_daily)
except Exception as exc:
    final_value = strategy_instance.broker.getvalue()
    print(f'Final portfolio value: {final_value:,.2f}')
    print(f'(Detailed metrics unavailable: {exc})')



RÉSULTATS DU BACKTEST

📊 Période: 2015-01-02 à 2024-12-31
📊 Nombre de bougies: 2516

💰 PERFORMANCE
   Capital Initial:              10,000.00 €
   Capital Final:                13,103.44 €
   P&L:                           3,103.44 € (+31.03%)
   Retour Total:                     27.03%
   Retour Moyen (annuel):             0.01%

📈 TRADES
   Nombre Total:                        33
   Trades Gagnants:                     20
   Trades Perdants:                     13
   Win Rate:                        60.61%
   Gain Moyen:                      301.95 €
   Perte Moyenne:                  -225.81 €

📉 RISQUE
   Sharpe Ratio:           0.4295479387595011
   Max Drawdown:                     8.59%



In [7]:
import pandas as pd

def _to_naive_timestamp(value):
    if value is None:
        return None
    ts = pd.Timestamp(value)
    if ts.tzinfo is not None:
        ts = ts.tz_convert('UTC').tz_localize(None)
    return ts

def extract_closed_trades(trade_analysis):
    closed = trade_analysis.get('closed', []) or []
    rows = []
    for trade in closed:
        entry_ts = _to_naive_timestamp(trade.get('open_datetime') or trade.get('dtopen'))
        exit_ts = _to_naive_timestamp(trade.get('close_datetime') or trade.get('dtclose'))
        rows.append({
            'entry_time': entry_ts,
            'exit_time': exit_ts,
            'entry_price': trade.get('open_price') or trade.get('price'),
            'exit_price': trade.get('close_price'),
            'size': trade.get('size', trade.get('opened')),
            'pnl': trade.get('pnl'),
        })
    return pd.DataFrame(rows)

trade_analysis = strategy_instance.analyzers.trades.get_analysis()
trades_df = extract_closed_trades(trade_analysis)

time_returns = strategy_instance.analyzers.timereturn.get_analysis()
if time_returns:
    equity_series = pd.Series(time_returns).sort_index()
    equity_curve = initial_capital * (1 + equity_series).cumprod()
else:
    equity_curve = pd.Series(dtype='float64')

if getattr(df_daily.index, 'tz', None) is not None:
    price_index = df_daily.index.tz_convert('UTC').tz_localize(None)
else:
    price_index = df_daily.index

price_lookup = df_daily[['close']].copy()
price_lookup.index = price_index

fig_bt = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])
fig_bt.add_trace(go.Candlestick(x=price_index, open=df_daily['open'], high=df_daily['high'], low=df_daily['low'], close=df_daily['close'], name='OHLC'), row=1, col=1)
fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['EMA_FAST'], mode='lines', name='EMA_FAST'), row=1, col=1)
fig_bt.add_trace(go.Scatter(x=price_index, y=df_plot['EMA_SLOW'], mode='lines', name='EMA_SLOW'), row=1, col=1)

if not trades_df.empty:
    entries = trades_df.dropna(subset=['entry_time'])
    entry_prices = entries['entry_price']
    if entry_prices.isna().any():
        fallback = price_lookup['close'].reindex(entries['entry_time'], method='nearest')
        entry_prices = entry_prices.fillna(fallback)
    fig_bt.add_trace(go.Scatter(x=entries['entry_time'], y=entry_prices, mode='markers', name='Entry', marker=dict(symbol='triangle-up', color='green', size=10)), row=1, col=1)
    exits = trades_df.dropna(subset=['exit_time'])
    if not exits.empty:
        exit_prices = exits['exit_price']
        if exit_prices.isna().any():
            fallback_exit = price_lookup['close'].reindex(exits['exit_time'], method='nearest')
            exit_prices = exit_prices.fillna(fallback_exit)
        fig_bt.add_trace(go.Scatter(x=exits['exit_time'], y=exit_prices, mode='markers', name='Exit', marker=dict(symbol='triangle-down', color='red', size=10)), row=1, col=1)

if not equity_curve.empty:
    eq_index = equity_curve.index
    if getattr(eq_index, 'tz', None) is not None:
        eq_index = eq_index.tz_convert('UTC').tz_localize(None)
    fig_bt.add_trace(go.Scatter(x=eq_index, y=equity_curve, mode='lines', name='Equity Curve'), row=2, col=1)
else:
    fig_bt.add_trace(go.Scatter(x=[price_index.min()], y=[initial_capital], mode='lines', name='Equity Curve'), row=2, col=1)

fig_bt.update_yaxes(title_text='Price', row=1, col=1)
fig_bt.update_yaxes(title_text='Equity', row=2, col=1)
fig_bt.update_layout(title='Interactive backtest review', template='plotly_white', xaxis_rangeslider_visible=False)
fig_bt.show()

trades_df.head()


""


## 4. Robustness Notes
- Walk-forward / out-of-sample checks
- Parameter sensitivity maps
- Monte Carlo trade sequencing
- Cost / slippage stress tests
